<a href="https://colab.research.google.com/github/souzamichel/ml_trading_test_code/blob/main/ml_trading_backtest_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.ensemble import RandomForestClassifier
from itertools import product

In [11]:
# Função para calcular RSI
def compute_rsi(series, period=14):
    delta    = series.diff()
    gain     = delta.clip(lower=0)
    loss     = -delta.clip(upper=0)
    avg_gain = gain.rolling(period).mean()
    avg_loss = loss.rolling(period).mean()
    rs       = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

In [12]:
# Métricas adicionais
def calculate_metrics(returns):
    cumulative = (1 + returns).cumprod() - 1
    total_ret  = cumulative.iloc[-1]
    ann_vol    = returns.std() * np.sqrt(252)
    sharpe     = returns.mean() / returns.std() * np.sqrt(252) if returns.std() else 0.0

    # Drawdown máximo
    running_max    = cumulative.cummax()
    drawdown       = cumulative - running_max
    max_drawdown   = drawdown.min()

    # Fator de recuperação
    recovery = total_ret / abs(max_drawdown) if max_drawdown < 0 else np.nan

    # Win rate
    win_rate = (returns > 0).sum() / len(returns)

    return {
        "Cumulative Return": total_ret,
        "Volatility": ann_vol,
        "Sharpe": sharpe,
        "Max Drawdown": max_drawdown,
        "Recovery Factor": recovery,
        "Win Rate": win_rate
    }

In [13]:
# Grades de parâmetros
param_grid = {
    "ma_short":   [5, 10, 15],
    "ma_long":    [10, 20, 30],
    "rsi_period": [14, 21],
    "mom_period": [5, 10, 20]
}
rf_grid = {
    "n_estimators": [50, 100],
    "max_depth":    [None, 5, 10]
}

def combos(grid):
    for vals in product(*grid.values()):
        yield dict(zip(grid.keys(), vals))

In [14]:
# Download de dados
symbol = "AAPL"
data = yf.download(symbol, period="5y", interval="1d",
                   auto_adjust=True, progress=False)

results = []

In [15]:

# Laço de backtest
for ind in combos(param_grid):
    df = data.copy()
    # Indicadores
    df["MA_short"] = df["Close"].rolling(ind["ma_short"]).mean()
    df["MA_long"]  = df["Close"].rolling(ind["ma_long"]).mean()
    df["RSI"]      = compute_rsi(df["Close"], ind["rsi_period"])
    df["Mom"]      = df["Close"].pct_change(ind["mom_period"])
    df.dropna(inplace=True)

    # Retorno diário
    df["Return"] = df["Close"].pct_change()
    df.dropna(inplace=True)

    # Target binário
    fut = df["Close"].pct_change().shift(-1)
    df["Target"] = (fut > 0).astype(int)
    df.dropna(inplace=True)

    X   = df[["MA_short","MA_long","RSI","Mom"]]
    y   = df["Target"]
    ret = df["Return"]

    split = int(len(df) * 0.8)
    X_tr, X_te = X.iloc[:split], X.iloc[split:]
    y_tr, y_te = y.iloc[:split], y.iloc[split:]
    ret_te     = ret.iloc[split:]

    if X_te.shape[0] < 2 or y_tr.nunique() < 2:
        continue

    for rf_params in combos(rf_grid):
        model = RandomForestClassifier(**rf_params, random_state=42)
        model.fit(X_tr, y_tr)

        preds = model.predict(X_te)
        df_sig = pd.DataFrame({"signal": preds}, index=X_te.index)
        df_ret = pd.DataFrame({"return": ret_te}, index=ret_te.index)

        df_test = (
            df_sig.join(df_ret, how="inner")
                  .assign(signal=lambda d: d["signal"].shift(1, fill_value=0))
        )
        df_test["strat"] = df_test["signal"] * df_test["return"]
        if df_test["strat"].empty:
            continue

        # Calcula métricas
        m = calculate_metrics(df_test["strat"])
        results.append({**ind, **rf_params, **m})

In [16]:
# Organiza resultados
results_df = pd.DataFrame(results)
if results_df.empty:
    print("Nenhum backtest válido encontrado.")
else:
    # Top 10 por Sharpe
    top10 = results_df.sort_values("Sharpe", ascending=False).head(10)

    for idx, row in top10.iterrows():
        print(f"\nEstratégia #{idx+1}")
        print(f"Médias: short={row['ma_short']}, long={row['ma_long']}; RSI={row['rsi_period']}, Mom={row['mom_period']}")
        print(f"RandomForest: n_estimators={row['n_estimators']}, max_depth={row['max_depth']}")
        print(f"Retorno Acumulado: {row['Cumulative Return']:.2%}  → mostra o ganho total da estratégia.")
        print(f"Volatilidade Anual: {row['Volatility']:.2%}  → risco medido em desvio padrão anualizado.")
        print(f"Sharpe Ratio: {row['Sharpe']:.2f}  → retorno ajustado pelo risco.")
        print(f"Drawdown Máximo: {row['Max Drawdown']:.2%}  → maior perda desde o pico ao fundo.")
        print(f"Fator de Recuperação: {row['Recovery Factor']:.2f}  → relação entre retorno total e drawdown máximo.")
        print(f"Win Rate: {row['Win Rate']:.2%}  → proporção de trades vencedores.")


Estratégia #75
Médias: short=5.0, long=30.0; RSI=14.0, Mom=5.0
RandomForest: n_estimators=50.0, max_depth=10.0
Retorno Acumulado: 37.46%  → mostra o ganho total da estratégia.
Volatilidade Anual: 18.42%  → risco medido em desvio padrão anualizado.
Sharpe Ratio: 1.86  → retorno ajustado pelo risco.
Drawdown Máximo: -11.22%  → maior perda desde o pico ao fundo.
Fator de Recuperação: 3.34  → relação entre retorno total e drawdown máximo.
Win Rate: 30.08%  → proporção de trades vencedores.

Estratégia #181
Médias: short=10.0, long=30.0; RSI=14.0, Mom=5.0
RandomForest: n_estimators=50.0, max_depth=nan
Retorno Acumulado: 43.89%  → mostra o ganho total da estratégia.
Volatilidade Anual: 22.61%  → risco medido em desvio padrão anualizado.
Sharpe Ratio: 1.76  → retorno ajustado pelo risco.
Drawdown Máximo: -12.42%  → maior perda desde o pico ao fundo.
Fator de Recuperação: 3.53  → relação entre retorno total e drawdown máximo.
Win Rate: 28.05%  → proporção de trades vencedores.

Estratégia #18